In [ ]:
# import yahoo finance data
import yfinance as yf
# import stockstats data
from stockstats import StockDataFrame as ss

# import necessary libraries
import matplotlib as mp
import numpy as np
import pandas as pd
import pytz
import warnings
import time
import random
import statistics
import pydoc
import os
import pyarrow
import pandas_gbq
import statsmodels
import tensorflow

#import libraries for SARIMA model
import pmdarima as pm
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pickle
from datetime import datetime
from datetime import timedelta
from datetime import date

Download Data for Apple Stock

In [ ]:
apple_ticker = yf.Ticker("AAPL")
apple_data = yf.download("AAPL", start = '2020-01-01', interval = '1d')
apple_df = ss.retype(apple_data)

apple_data[['stochrsi', 'macd', 'mfi']] = apple_df[['stochrsi', 'macd', 'mfi']]
print(apple_data)
print(apple_ticker.get_capital_gains)

Load Data Into SQL -> BigQuery

In [ ]:
SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/drive',
]

# import google cloud service account and bigquery
from google.oauth2 import service_account
from google.cloud import bigquery

# specify google cloud project information
credentials = service_account.Credentials.from_service_account_file(
    'black-vehicle-406619-bf2e31773163.json')
project_id = 'black-vehicle-406619'
client = bigquery.Client(project=project_id, credentials=credentials)
dataset_id = 'stocks_ds'
table_id = '20yrs_stockdata'
table_path = f"{project_id}.{dataset_id}.{table_id}"

# specify load reqs
load_info = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
load_data = client.load_table_from_dataframe(apple_data, table_path, job_config=load_info)
load_data.result()

Preparing Data: SARIMA Model

- Using indexing to remove duplicates, therefore preventing bias and skewing the distrubotion of the data.
- Training the model on close prices only.

In [ ]:
#Seasonal - fit stepwise auto-ARIMA
#!pip install pmdarima

# Remove any duplicate index
apple_data = apple_data.loc[~apple_data.index.duplicated(keep='first')]

#Filter only required data

apple_data = apple_data[['Close']]





Preparing Data: SARIMA Model
- Created an instance of MinMaxScaler class to use while creating a new DataFrame.
- We are scaling Apple's stock prices to a range between 0 and 1 for the LTSM model. By doing so, we are improving learning stability and the speed of the model, also creating numerical stability.

- Feature_Scaler.fit_transform(apple_data) scales the 'Close' column of the apple_data DataFrame using the fit_transform method. This method fits the scaler to the data and transforms it in a single step.
- Used np.squeeze to reshape the result from the scaler, removing any unnecessary dimensions.
- Created DataFrame "apple_transformed", while specifying the column to be named Close and ensure the index matches the original DataFrame.

In [ ]:
#Scale the APPL data into a standard range using MinMaxScaler ()

Feature_Scaler = MinMaxScaler()

#Transform current APPL data

apple_transformed = pd.DataFrame(np.squeeze(Feature_Scaler.fit_transform(apple_data), axis=1), columns=["Close"], index=apple_data.index)


Creating SARIMA Model
- `pm.auto_arima` function is used to fit a Seasonal ARIMA model to the 'Close' price data of Apple stock.

In [ ]:

sarima_model = pm.auto_arima(apple_transformed["Close"], start_p=1, start_q=1,
                         test='adf',
                         max_p=3, max_q=3,
                         m=12, #12 is the frequency of the cycle (yearly)
                         start_P=0,
                         seasonal=True, #set to seasonal
                         d=None,
                         D=1, #order of the seasonal differencing
                         trace=False,
                         error_action='ignore',
                         suppress_warnings=True,
                         stepwise=True)

# start_p=1, start_q=1: Sets the initial values for the order of the AR (AutoRegressive) and MA (Moving Average) components in the non-seasonal part of the model.
# test='adf': Specifies the use of the Augmented Dickey-Fuller (ADF) test to determine whether the time series is stationary and to help in determining the need for differencing (`d` parameter).
# max_p=3, max_q=3: Specifies the maximum values for the `p` and `q` parameters to consider during the model fitting process.
# start_P=0: Sets the initial value for the order of the seasonal AR component.
# d=None: The order of non-seasonal differencing is not specified, which allows the function to determine it automatically.
# trace=False: This means that the function will not print out diagnostic information about the steps it's taking.
# error_action='ignore': Instructs the function to ignore errors and try different combinations of parameters.
# suppress_warnings=True: Suppresses convergence warnings, which can be frequent in ARIMA modeling.
# stepwise=True: Enables a stepwise search to efficiently find the best model parameters.

SARIMA Model: Create diagnostic plots
- Residual Plot
- Histrogram Plot
- Normal Q-Q Plot
- Correlogram

In [ ]:
sarima_model.plot_diagnostics(figsize=(15,12))
plt.show()

SARIMA Model: Serialize with `Pickle`
- Serializing = Converting the model object into a format that can be easily stored in a file for later use or sharing without having to retrain.
- Pickle allows the use of 'wb' and 'rb' modes to open a file for reading or writing in binary mode.

In [ ]:
# Serialize with Pickle and save it as pkl
with open('sarima_model.pkl', 'wb') as pkl:
    pickle.dump(sarima_model, pkl)

# Desiarilize the content of the file back into a Python object
with open('sarima_model.pkl', 'rb') as pkl:
    loaded_model = pickle.load(pkl)

SARIMA Model: Predicting APPL Stock Price
- Define a function named forecast that takes a time series model, a DataFrame (df), and a forecast date as input, and returns a DataFrame containing actual values, predicted values, and confidence intervals.

In [ ]:
def forecast(model, df, forecast_date):
    # Forecast
    n_periods = (datetime.strptime(forecast_date, '%Y-%m-%d') - df.index[-1]).days
    fitted, confint = model.predict(n_periods=n_periods, return_conf_int=True)
    index_of_fc = pd.date_range(df.index[-1] + pd.DateOffset(days=1), periods = n_periods, freq='D')

    # Make series for plotting purpose
    fitted_series = pd.Series(fitted.values, index=index_of_fc)
    lower_series = pd.Series(confint[:, 0], index=index_of_fc)
    upper_series = pd.Series(confint[:, 1], index=index_of_fc)

    #Concatenate the original DataFrame with forecasted values and confidence intervals
    df_result = pd.concat([df, fitted_series, lower_series, upper_series], axis=1)
    df_result.columns = ["Actual", "Prediction", "Low", "High"]

    #Inverse transform the scaled values to their original scale
    for column in df_result.columns:
      df_result[column] = Feature_Scaler.inverse_transform(df_result[column].values.reshape(-1,1))


    return df_result

SARIMA Model: Store APPL Stock Price
- Use the above defined function to store your predicted values in a `test` variable
- View what is stored in `test` variable
- Plot the range of predicted prices for APPL stock

In [ ]:
test=forecast(loaded_model, apple_transformed, "2023-12-20")

In [ ]:
test

In [ ]:
# Plot
plt.figure(figsize=(15,7))
plt.plot(test["Actual"][-120:], color='#1f76b4')
plt.plot(test["Prediction"], color='darkgreen')
plt.fill_between(test.index,
                test["Low"],
                test["High"],
                color='k', alpha=.15)

plt.title("SARIMA - Forecast of APPL Stock Price")
plt.show()

Load Predicted Data into BigQuery

In [ ]:

# specify google cloud project information
dataset_id = 'predicted_prices'
table_id = 'SARIMA and LTSM Predicted Prices'
table_path = f"{project_id}.{dataset_id}.{table_id}"

# specify load reqs
load_info = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
# enter combined data load_data = client.load_table_from_dataframe(combined data, table_path, job_config=load_info)
load_data.result()